In [1]:
%pip install shapely

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from ultralytics import YOLO
from shapely.geometry import Polygon, box

from typing import NamedTuple

class GameSquare(NamedTuple):
    id: int
    name: str
    points: list
    center: tuple
    color: str
    contains_dragon: bool
    next_square: int

def get_center(points):
    x_coords = [point[0] for point in points]
    y_coords = [point[1] for point in points]
    center_x = sum(x_coords) / len(x_coords)
    center_y = sum(y_coords) / len(y_coords)
    return (center_x, center_y) 


class ReferenceMap:
    def __init__(self, reference_image_path, annotations_path):
        self.reference_image = cv2.imread(reference_image_path)
        self.reference_squares = self._load_annotations(annotations_path)
        self.reference_corners = np.float32([
            [0, 0],               # top-left
            [2883, 0],           # top-right
            [2883, 2550],        # bottom-right
            [0, 2550]            # bottom-left
        ])

    def _load_annotations(self, annotations_path):
            """
            Load square coordinates from XML annotations
            """
            tree = ET.parse(annotations_path)
            root = tree.getroot()
            
            coordinates = {}
            self.original_boxes = {}  # Store original box information
            
            game_squares = []

            for idx, polygon in enumerate(root.findall(".//polygon")):

                label = polygon.get('label')
                points = polygon.get('points')
                
        # Convert points string to list of coordinates
                # Points are typically stored as "x1,y1;x2,y2;x3,y3;..."
                point_pairs = points.split(';')
                coordinates = []
                for pair in point_pairs:
                    x, y = map(float, pair.split(','))
                    coordinates.append((x, y))
                square_color = label.split("_")[0]
                square_name = f'{idx}_{label}'
                square_center = get_center(coordinates)
                if square_name == "2_Yellow_Square":
                    next_square = 33
                elif square_name == "17_Green_square":
                    next_square = 27
                elif label == "Final_Square":
                    next_square == idx
                else:
                    next_square = idx+1
                contains_dragon = "Dragon" in square_name
                game_square = GameSquare(idx, square_name, coordinates, square_center, square_color, contains_dragon, next_square)
                game_squares.append(game_square)

            return game_squares
    

    def show_map_with_annotations(self):
        """
        Display the reference map with annotated coordinates.
        """
        img_copy = self.reference_image.copy()
        
        for square in self.reference_squares:
            # Get the points for the current square
            points = square.points  # Access the points from the GameSquare
            
            # Draw the polygon for the square
            cv2.polylines(img_copy, [np.int32(points)], isClosed=True, color=(255, 255, 255), thickness=4)  # Green polygon
            
            # Draw the annotated coordinates on the image
            cv2.circle(img_copy, (int(square.center[0]), int(square.center[1])), 5, (0, 255, 0), -1)  # Green dot for coordinates
            
            # Add label for the square
            cv2.putText(img_copy, square.name, (int(square.center[0]), int(square.center[1]) - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
        
        cv2.imshow('Annotated Reference Map', img_copy)
        cv2.waitKey(0)
        cv2.destroyAllWindows()



In [3]:
refMap = ReferenceMap('ReferenceMap.png', 'candylandMapAnnotations.xml')

In [4]:
refMap.reference_squares

[GameSquare(id=0, name='0_Red_Square', points=[(635.4, 2274.77), (621.45, 2411.99), (795.87, 2416.64), (802.85, 2281.75)], center=(713.8924999999999, 2346.2875), color='Red', contains_dragon=False, next_square=1),
 GameSquare(id=1, name='1_Purple_Square', points=[(811.65, 2283.22), (809.71, 2415.01), (982.2, 2411.14), (972.51, 2279.35)], center=(894.0175000000002, 2347.18), color='Purple', contains_dragon=False, next_square=2),
 GameSquare(id=2, name='2_Yellow_Square', points=[(982.2, 2279.35), (991.89, 2416.95), (1170.19, 2385.94), (1145.0, 2248.34)], center=(1072.3200000000002, 2332.645), color='Yellow', contains_dragon=False, next_square=33),
 GameSquare(id=3, name='3_Blue_Square', points=[(1150.81, 2248.34), (1177.94, 2382.06), (1336.86, 2347.18), (1319.42, 2217.33)], center=(1246.2575, 2298.7275), color='Blue', contains_dragon=False, next_square=4),
 GameSquare(id=4, name='4_Orange_Square', points=[(1329.11, 2215.39), (1346.55, 2349.12), (1499.66, 2325.86), (1488.03, 2194.07)], ce

In [99]:
class GameBoard:

    def __init__(self, reference_image_path, annotations_path):
        """
        Initialize with reference image and XML annotations
        
        Args:
            reference_image_path: Path to the reference Candyland board image
            annotations_path: Path to the XML file containing square coordinates
        """

        self.reference_map = ReferenceMap(reference_image_path, annotations_path)
        
        # Store reference corner coordinates (clockwise from top-left)
        self.corner_detector = YOLO('corner_detector.pt')
        self.piece_detector = YOLO('game_piece_detector.pt')
        self.pieces = {}

    def _detect_corners(self, camera_image):
        results = self.corner_detector.predict(camera_image)
        corners = results[0].boxes.xyxy.numpy()
        
        # Calculate centers for each detected corner box
        centers = []
        for box in corners:
            x1, y1, x2, y2 = box
            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2
            centers.append([center_x, center_y])
        
        centers = np.float32(centers)
        if len(centers) != 4:
            raise ValueError("Error: Detected corners do not match expected number of corners")
        # Sort corners: top-left, top-right, bottom-right, bottom-left
        centers = sorted(centers, key=lambda point: (point[1], point[0]))  # Sort by y first, then x
        top_left, top_right = sorted(centers[:2], key=lambda point: point[0])  # Sort top two by x
        bottom_left, bottom_right = sorted(centers[2:], key=lambda point: point[0])  # Sort bottom two by x
        
        ordered_corners = np.array([top_left, top_right, bottom_right, bottom_left])
        print("ordered corners", ordered_corners)
        return ordered_corners    
    


    def _get_game_square_coordinates(self, camera_image):
        """
        Get the coordinates of all game squares in the camera image
        
        Args:
            camera_image: Current frame from the overhead camera
        Returns:
            transformed_coordinates: Dictionary mapping square IDs to their box coordinates
        """
        # Hardcoded camera image corners (clockwise from top-left)
        camera_corners = self._detect_corners(camera_image)
        
        # Calculate homography
        H, _ = cv2.findHomography(self.reference_map.reference_corners, camera_corners)
        
        # Transform all reference coordinates to camera coordinates
        camera_squares = []
        for square in self.reference_map.reference_squares:
            polygon = square.points
            
            # Convert polygon points to homogeneous coordinates
            square_points =  np.float32([[x, y, 1] for x, y in polygon])
            
            # Transform each point using homography
            transformed_points = []
            for point in square_points:
                transformed_point = np.dot(H, point)
                transformed_point = transformed_point / transformed_point[2]
                transformed_points.append(transformed_point[:2])
            transformed_center = get_center(transformed_points)
            camera_square = GameSquare(
                id= square.id,
                name= square.name,
                points= transformed_points,
                center= transformed_center,
                color= square.color,
                contains_dragon=square.contains_dragon,
                next_square= square.next_square
            )

            camera_squares.append(camera_square)
            

        return camera_squares
    

    
    def _piece_in_square(self, piece_box, square):
        
        return True
    
    def _detect_pieces(self, camera_image):
        results = self.piece_detector.predict(camera_image)
        piece_classes = results[0].boxes.cls.numpy()
        names = results[0].names
        piece_names = [names[int(cls)] for cls in piece_classes]
        piece_boxes = results[0].boxes.xyxy.numpy()
        return dict(zip(piece_names, piece_boxes))
    



    
    def update_game_board(self, camera_image):
        self.latest_game_frame = camera_image
        self.corners = self._detect_corners(camera_image)
        self.game_squares = self._get_game_square_coordinates( camera_image)
        self.pieces = self._detect_pieces(camera_image)
        print("pieces", self.pieces)

    def show_game_board_with_annotations(self):
        """
        Visualize boxes on the image with labels
        
        Args:
            image: Image to draw on
            transformed_coordinates: Dictionary mapping square IDs to their corner coordinates
            color: BGR color tuple (default: red)
            thickness: Line thickness in pixels
        Returns:
            Image with boxes and labels drawn
        """
        annotated_frame = self.annotate_frame(self.latest_game_frame)

        cv2.imshow('Annotated Gameboard', annotated_frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def annotate_frame(self, frame):
        img_copy = frame.copy()
        for square in self.game_squares:
            # Get the points for the current square
            points = square.points  # Access the points from the GameSquare
            
            red = (0, 0, 255)
            purple = (255, 0, 255)
            yellow = (0, 255, 255)  
            blue = (255, 0, 0)
            orange = (0, 165, 255)
            green = (0, 255, 0)

            if square.color == "Yellow":
                color = yellow
            elif square.color == "Green":
                color = green
            elif square.color == "Blue":
                color = blue
            elif square.color == "Purple":
                color = purple
            elif square.color == "Orange":
                color = orange  
            elif square.color == "Red":
                color = red
            else:
                color = (255,255,255)

            if square.contains_dragon:
                text = square.name
            elif square.color not in ["Yellow", "Green", "Purple", "Orange", "Red", "Blue"]:
                text = square.name
            else:
                text = str(square.id)
            # Draw the polygon outline in white
            cv2.polylines(img_copy, [np.int32(points)], isClosed=True, color=(0, 0, 0), thickness=10)  # White outline
            
            # Draw the polygon interior in the specified color
            cv2.polylines(img_copy, [np.int32(points)], isClosed=True, color=color, thickness=4)  # Colored interior            
            # Draw the annotated coordinates on the image
            cv2.circle(img_copy, (int(square.center[0]), int(square.center[1])), 5, green, -1)  # Green dot for coordinates
            
            # Draw a black rectangle behind the text for highlighting
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)[0]
            text_x = int(square.center[0]) - text_size[0] // 2
            text_y = int(square.center[1]) - 10 - text_size[1] // 2
            cv2.rectangle(img_copy, (text_x - 5, text_y - text_size[1] - 5), 
                           (text_x + text_size[0] + 5, text_y + 5), (0, 0, 0), -1)  # Black rectangle
            
            cv2.putText(img_copy, text, (text_x, text_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)            
        
        for piece, box in self.pieces.items():      

            cv2.rectangle(img_copy, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255, 255, 255), 10)  # Fixed alignment by removing the stray '0'
            text_size = cv2.getTextSize(str(piece), cv2.FONT_HERSHEY_SIMPLEX, 1, 1)[0]
            text_x = int(box[0]) + (int(box[2]) - int(box[0])) // 2 - text_size[0] // 2  # Centering the text horizontally
            text_y = int(box[1]) - 10 - text_size[1] // 2
            cv2.rectangle(img_copy, (text_x - 5, text_y - text_size[1] - 5), 
                           (text_x + text_size[0] + 5, text_y + 5), (0, 0, 0), -1)  # Black rectangle

            cv2.putText(img_copy, str(piece), (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)  # Updated to use text_x for alignment
        return img_copy

        


In [72]:
# Initialize the mapper
game_board = GameBoard('ReferenceMap.png', 'candylandMapAnnotations.xml')
camera_image_path = '../MockGamePieceDetection/RawImagesRound1/IMG_4769 2.JPG'

camera_image = cv2.imread(camera_image_path)
game_board.update_game_board(camera_image)




0: 640x640 4 Corners, 63.1ms
Speed: 4.8ms preprocess, 63.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
ordered corners [[     168.83      245.01]
 [     2778.1       135.5]
 [     2997.3      2481.6]
 [     99.394      2560.8]]

0: 640x640 4 Corners, 80.5ms
Speed: 3.5ms preprocess, 80.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
ordered corners [[     168.83      245.01]
 [     2778.1       135.5]
 [     2997.3      2481.6]
 [     99.394      2560.8]]

0: 640x640 1 Green Square, 1 Purple Square, 1 Striped Triangle, 1 Yellow Circle, 63.3ms
Speed: 2.8ms preprocess, 63.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
pieces {'Purple Square': array([     1840.9,       942.9,      1969.6,      1067.3], dtype=float32), 'Striped Triangle': array([     1796.6,      1813.8,      1905.1,      1935.3], dtype=float32), 'Green Square': array([     2334.4,      1760.8,      2478.9,      1910.9], dtype=float32), 'Yellow Circle': arra

In [73]:
game_board.show_game_board_with_annotations()

In [71]:
corners = mapper.detect_corners('ReferenceMap.png')
print(corners)


image 1/1 /Users/srikargudimella/Documents/Senior-Design/Game-Board-Mapping/ReferenceMap.png: 576x640 4 Corners, 83.5ms
Speed: 6.9ms preprocess, 83.5ms inference, 0.9ms postprocess per image at shape (1, 3, 576, 640)
[[  30.5391     32.618683]
 [  26.39428  2530.645   ]
 [2856.0447     28.403473]
 [  37.63608    40.39537 ]]


In [64]:
import cv2
import time

class GameBoardStream:
    def __init__(self, game_board, video_source=0, update_interval=2.0):
        """
        Initialize the game board stream
        
        Args:
            game_board: Instance of GameBoard class
            video_source: Camera index or video file path (default: 0 for primary camera)
            update_interval: Time between updates in seconds (default: 2.0)
        """
        self.game_board = game_board
        self.video_source = video_source
        self.update_interval = update_interval
        self.cap = None
        
    def start(self):
        """Start capturing and processing the video stream"""
        self.cap = cv2.VideoCapture(self.video_source)
        if not self.cap.isOpened():
            raise ValueError(f"Failed to open video source: {self.video_source}")
            
        last_update = 0
        
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    break
                    
                current_time = time.time()
                
                # Update game board state every update_interval seconds
                if current_time - last_update >= self.update_interval:
                    try:
                        self.game_board.update_game_board(frame)
                        print(f"Current frame number: {self.cap.get(cv2.CAP_PROP_POS_FRAMES)}")
                        self.game_board.show_game_board_with_annotations()
                        last_update = current_time
                    except ValueError as e:
                        print(f"Failed to update game board: {e}. Trying next frame...")
                        continue
                
                # Display the raw frame
                cv2.imshow('Raw Game Board Feed', frame)
                
                # Break loop on 'q' press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break                    
        finally:
            self.stop()
            
    def stop(self):
        """Clean up resources"""
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()

In [81]:
import cv2
import time

class GameBoardStream:
    def __init__(self, game_board, video_source=0, update_interval=2.0, output_path='output.mp4'):
        """
        Initialize the game board stream
        
        Args:
            game_board: Instance of GameBoard class
            video_source: Camera index or video file path (default: 0 for primary camera)
            update_interval: Time between updates in seconds (default: 2.0)
            output_path: Path where the annotated video will be saved
        """
        self.game_board = game_board
        self.video_source = video_source
        self.update_interval = update_interval
        self.output_path = output_path
        self.cap = None
        
    def start(self):
        """Process the video and create annotated output"""
        self.cap = cv2.VideoCapture(self.video_source)
        if not self.cap.isOpened():
            raise ValueError(f"Failed to open video source: {self.video_source}")
        
        # Get video properties
        fps = self.cap.get(cv2.CAP_PROP_FPS)
        frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(self.output_path, fourcc, fps, (frame_width, frame_height))
        
        last_update = 0
        freeze_frames = int(fps * 2)  # Number of frames to freeze (2 seconds)
        
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    break
                    
                current_time = time.time()
                
                # Update game board state every update_interval seconds
                if current_time - last_update >= self.update_interval:
                    try:
                        self.game_board.update_game_board(frame)
                        print(f"Current frame number: {self.cap.get(cv2.CAP_PROP_POS_FRAMES)}")
                        
                        # Create annotated frame
                        annotated_frame = self.game_board.annotate_frame(frame.copy())  # Capture the returned annotated frame
                        cv2.imshow('Annotated Gameboard', annotated_frame)
                        cv2.waitKey(0)
                        cv2.destroyAllWindows()

                        # Write regular frame
                        out.write(frame)
                        
                        # Write annotated frame multiple times to create freeze effect
                        for _ in range(freeze_frames):
                            out.write(annotated_frame)
                            
                        last_update = current_time
                    except ValueError as e:
                        print(f"Failed to update game board: {e}. Trying next frame...")
                        out.write(frame)  # Write original frame if annotation fails
                        continue
                else:
                    # Write regular frame
                    out.write(frame)
                
                # Display progress
                cv2.imshow('Processing Video', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            self.stop()
            out.release()
            
    def stop(self):
        """Clean up resources"""
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()

In [88]:
class GameBoardStream:
    def __init__(self, game_board, video_source=0, output_path='output.mp4', annotation_timestamps=None):
        """
        Initialize the game board stream
        
        Args:
            game_board: Instance of GameBoard class
            video_source: Camera index or video file path
            output_path: Path where the annotated video will be saved
            annotation_timestamps: List of timestamps (in seconds) where annotations should occur
        """
        self.game_board = game_board
        self.video_source = video_source
        self.output_path = output_path
        self.annotation_timestamps = sorted(annotation_timestamps) if annotation_timestamps else []
        self.cap = None
        
    def start(self):
        """Process the video and create annotated output"""
        self.cap = cv2.VideoCapture(self.video_source)
        if not self.cap.isOpened():
            raise ValueError(f"Failed to open video source: {self.video_source}")
        
        # Get video properties
        fps = self.cap.get(cv2.CAP_PROP_FPS)
        frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(self.output_path, fourcc, fps, (frame_width, frame_height))
        
        freeze_frames = int(fps * 2)  # Number of frames to freeze (2 seconds)
        current_timestamp_idx = 0
        
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    break
                
                current_frame = self.cap.get(cv2.CAP_PROP_POS_FRAMES)
                current_time = current_frame / fps
                
                # Check if we need to annotate at this timestamp
                should_annotate = (
                    current_timestamp_idx < len(self.annotation_timestamps) and 
                    abs(current_time - self.annotation_timestamps[current_timestamp_idx]) < 1/fps
                )
                
                if should_annotate:
                    try:
                        self.game_board.update_game_board(frame)
                        print(f"Annotating at timestamp: {current_time:.2f}s")
                        
                        # Create annotated frame
                        annotated_frame = self.game_board.annotate_frame(frame.copy())
                        
                        # Optional: Display the annotated frame
                        cv2.imshow('Annotated Gameboard', annotated_frame)
                        cv2.waitKey(0)
                        cv2.destroyAllWindows()

                        # Write regular frame
                        out.write(frame)
                        
                        # Write annotated frame multiple times to create freeze effect
                        for _ in range(freeze_frames):
                            out.write(annotated_frame)
                            
                        current_timestamp_idx += 1
                    except ValueError as e:
                        print(f"Failed to annotate at {current_time:.2f}s: {e}")
                        out.write(frame)
                else:
                    # Write regular frame
                    out.write(frame)
                
                # Display progress
                progress = (current_frame / total_frames) * 100
                print(f"\rProgress: {progress:.1f}%", end="")
                
                cv2.imshow('Processing Video', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            self.stop()
            out.release()
            print("\nProcessing complete!")

In [100]:
class GameBoardStream:
    def __init__(self, game_board, video_source=0, output_path='output.mp4', annotation_timestamps=None):
        """
        Initialize the game board stream
        
        Args:
            game_board: Instance of GameBoard class
            video_source: Camera index or video file path
            output_path: Path where the annotated video will be saved
            annotation_timestamps: List of timestamps (in seconds) where annotations should occur
        """
        self.game_board = game_board
        self.video_source = video_source
        self.output_path = output_path
        self.annotation_timestamps = sorted(annotation_timestamps) if annotation_timestamps else []
        self.cap = None
        
    def start(self):
        """Process the video and create annotated output"""
        self.cap = cv2.VideoCapture(self.video_source)
        if not self.cap.isOpened():
            raise ValueError(f"Failed to open video source: {self.video_source}")
        
        # Get video properties
        fps = self.cap.get(cv2.CAP_PROP_FPS)
        frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(self.output_path, fourcc, fps, (frame_width, frame_height))
        
        current_timestamp_idx = 0
        max_retries = 30  # Maximum number of frames to try before skipping a timestamp
        
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    break
                
                current_frame = self.cap.get(cv2.CAP_PROP_POS_FRAMES)
                current_time = current_frame / fps
                
                # Check if we need to annotate at this timestamp
                should_annotate = (
                    current_timestamp_idx < len(self.annotation_timestamps) and 
                    current_time >= self.annotation_timestamps[current_timestamp_idx]
                )
                
                if should_annotate:
                    retry_count = 0
                    success = False
                    
                    # Keep trying frames until we get a successful update or hit max retries
                    while retry_count < max_retries and not success:
                        try:
                            self.game_board.update_game_board(frame)
                            print(f"\nSuccessfully annotated at timestamp: {current_time:.2f}s")
                            
                            # Create annotated frame
                            annotated_frame = self.game_board.annotate_frame(frame.copy())
                            
                            # Optional: Display the annotated frame
                            cv2.imshow('Annotated Gameboard', annotated_frame)
                            cv2.waitKey(0)
                            cv2.destroyAllWindows()

                            # Write regular frame
                            out.write(frame)
                            freeze_frames = int(fps * 15) if current_timestamp_idx == 0 else int(fps * 5)
                            # Write annotated frame multiple times to create freeze effect
                            for _ in range(freeze_frames):
                                out.write(annotated_frame)
                                
                            success = True
                            current_timestamp_idx += 1
                            
                        except ValueError as e:
                            retry_count += 1
                            print(f"\rFailed attempt {retry_count}/{max_retries} at {current_time:.2f}s: {e}", end="")
                            
                            # Try to get next frame
                            ret, frame = self.cap.read()
                            if not ret:
                                break
                            current_frame = self.cap.get(cv2.CAP_PROP_POS_FRAMES)
                            current_time = current_frame / fps
                    
                    if not success:
                        print(f"\nSkipping timestamp {self.annotation_timestamps[current_timestamp_idx]}s after {max_retries} failed attempts")
                        current_timestamp_idx += 1
                        out.write(frame)
                else:
                    # Write regular frame
                    out.write(frame)
                
                # Display progress
                progress = (current_frame / total_frames) * 100
                print(f"\rProgress: {progress:.1f}%", end="")
                
                cv2.imshow('Processing Video', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            self.stop()
            out.release()
            print("\nProcessing complete!")

    def stop(self):
        """Clean up resources"""
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()

In [102]:
game_board = GameBoard('ReferenceMap.png', 'candylandMapAnnotations.xml')
stream = GameBoardStream(game_board, video_source='/Users/srikargudimella/Downloads/IMG_5272.MOV', annotation_timestamps=[1.0, 4.0])
stream.start()

Progress: 19.9%
0: 576x640 5 Corners, 57.8ms
Speed: 1.9ms preprocess, 57.8ms inference, 0.9ms postprocess per image at shape (1, 3, 576, 640)
Failed attempt 1/30 at 1.00s: Error: Detected corners do not match expected number of corners
0: 576x640 5 Corners, 68.8ms
Speed: 1.6ms preprocess, 68.8ms inference, 0.4ms postprocess per image at shape (1, 3, 576, 640)
Failed attempt 2/30 at 1.03s: Error: Detected corners do not match expected number of corners
0: 576x640 5 Corners, 63.3ms
Speed: 1.6ms preprocess, 63.3ms inference, 0.4ms postprocess per image at shape (1, 3, 576, 640)
Failed attempt 3/30 at 1.07s: Error: Detected corners do not match expected number of corners
0: 576x640 4 Corners, 44.2ms
Speed: 1.7ms preprocess, 44.2ms inference, 0.4ms postprocess per image at shape (1, 3, 576, 640)
ordered corners [[     137.76      48.307]
 [     1205.3      29.512]
 [     1251.2      992.28]
 [      90.52      978.51]]

0: 576x640 4 Corners, 37.6ms
Speed: 1.6ms preprocess, 37.6ms inference, 